In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta
import time
import concurrent.futures   #多執行序

In [2]:
urls = []
date = datetime.strptime("2016/12/01", "%Y/%m/%d")   #把字串轉成日期
delta = timedelta(days = 1)

while date <= datetime.strptime(time.strftime("%Y/%m/%d"), "%Y/%m/%d"):  # time.strftime("%Y/%m/%d")現在日期時間
    resp = requests.get("https://udn.com/news/archive/2/6638/%s/1"%date.strftime("%Y/%m/%d"))  #把時分秒去掉
    resp = BeautifulSoup(resp.text,'html5lib')
    r = resp.select('div.pagelink > span.total')
    s = r[0].text                           
    r = re.findall('共 (.) 頁',s)[0]
    r = int(r)
    for p in range(1,r+1):
        resp = requests.get("https://udn.com/news/archive/2/6638/{}/{}".format(date.strftime("%Y/%m/%d"),p))
        resp = BeautifulSoup(resp.text,'html5lib')
        for url in resp.select('div#ranking_body > table#ranking_table > tbody > tr > td > a'):
                urls.append('https://udn.com' + url.get('href'))
        print("finish {} {}".format(date.strftime("%Y/%m/%d"),p))       

    date += delta

finish 2016/12/01 1
finish 2016/12/02 1
finish 2016/12/03 1
finish 2016/12/04 1
finish 2016/12/05 1
finish 2016/12/05 2
finish 2016/12/06 1
finish 2016/12/07 1
finish 2016/12/07 2
finish 2016/12/08 1
finish 2016/12/08 2
finish 2016/12/09 1
finish 2016/12/10 1
finish 2016/12/11 1
finish 2016/12/12 1
finish 2016/12/13 1
finish 2016/12/14 1
finish 2016/12/14 2
finish 2016/12/15 1
finish 2016/12/15 2
finish 2016/12/16 1
finish 2016/12/17 1
finish 2016/12/18 1
finish 2016/12/19 1
finish 2016/12/20 1
finish 2016/12/21 1
finish 2016/12/22 1
finish 2016/12/23 1
finish 2016/12/23 2
finish 2016/12/24 1
finish 2016/12/25 1
finish 2016/12/26 1
finish 2016/12/27 1
finish 2016/12/28 1
finish 2016/12/29 1
finish 2016/12/30 1
finish 2016/12/31 1
finish 2017/01/01 1
finish 2017/01/02 1
finish 2017/01/03 1
finish 2017/01/04 1
finish 2017/01/04 2
finish 2017/01/05 1
finish 2017/01/05 2
finish 2017/01/06 1
finish 2017/01/06 2
finish 2017/01/07 1
finish 2017/01/08 1
finish 2017/01/09 1
finish 2017/01/09 2


finish 2017/07/05 2
finish 2017/07/05 3
finish 2017/07/06 1
finish 2017/07/06 2
finish 2017/07/06 3
finish 2017/07/07 1
finish 2017/07/07 2
finish 2017/07/08 1
finish 2017/07/09 1
finish 2017/07/09 2
finish 2017/07/10 1
finish 2017/07/10 2
finish 2017/07/11 1
finish 2017/07/11 2
finish 2017/07/11 3
finish 2017/07/12 1
finish 2017/07/12 2
finish 2017/07/12 3
finish 2017/07/13 1
finish 2017/07/13 2
finish 2017/07/13 3
finish 2017/07/14 1
finish 2017/07/14 2
finish 2017/07/14 3
finish 2017/07/15 1
finish 2017/07/15 2
finish 2017/07/16 1
finish 2017/07/16 2
finish 2017/07/17 1
finish 2017/07/17 2
finish 2017/07/17 3
finish 2017/07/18 1
finish 2017/07/18 2
finish 2017/07/18 3
finish 2017/07/19 1
finish 2017/07/19 2
finish 2017/07/19 3
finish 2017/07/20 1
finish 2017/07/20 2
finish 2017/07/20 3
finish 2017/07/21 1
finish 2017/07/21 2
finish 2017/07/21 3
finish 2017/07/22 1
finish 2017/07/22 2
finish 2017/07/23 1
finish 2017/07/23 2
finish 2017/07/24 1
finish 2017/07/24 2
finish 2017/07/25 1


In [3]:
data_text = []
def get_detail_page(url):
    resp = requests.get(url)
    s = BeautifulSoup(resp.text, 'html5lib')
    articles = {}
    articles["id"] = url.split('/')[6]
    articles["title"] = s.select('div#story_body_content > h1.story_art_title')[0].text
    articles["media"] = 'united_daily_news'
    articles["url"] = url
    articles["category"] = "politics"
    
    
    if s.select('div.story_bady_info_author > a'):
        articles["reporter"] = s.select('div.story_bady_info_author > a')[0].text
    else:
        s.select('div.story_bady_info_author > a') == ''
            
    if re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text):
            resp = re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text)
            articles["location"] = resp[0][1]        
    else:
        re.findall('(.+)／(.+)報導',s.select("div.story_bady_info_author")[0].text) == ''
        
        
    articles["datetime"] = s.select('div.story_bady_info_author > span')[0].text.split()[0]
    
    text = ''
    for article in s.findAll('p'):
        text+=article.text
    articles["content"] = text
    
    articles["hash"] = hash("text")
    
    data_text.append(articles)
    
    datecsv = articles["datetime"].split("-")[0] + articles["datetime"].split("-")[1]
    
    with open('./Downloads/newspaper/United_Newspaper_text_{}.csv'.format(datecsv), 'a',encoding="utf-8")as f:    #將UTF8編碼改成中文
        f.write(json.dumps(data_text, ensure_ascii=False))
    
    print("ok")

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
    executor.map(get_detail_page, urls)

ok
okokokokokok





ok
okok

ok
ok
okokokokokokokok







okokokok
okok




ok
ok
okokokokok




ok
okokokok



ok
ok
okok

ok
okokokok



okok

okok

okok

okok
okokokokokokok







ok
ok
okokokokokokokokok








ok
ok
okok

ok
okok

ok
okok

ok
ok
ok
ok
ok
okok

okokok

ok

ok
okok

ok
ok
ok
okokok


ok
okokokokokok





ok
ok
ok
ok
okokokok



ok
ok
ok
ok
ok
okok
ok

ok
okok

okok

ok
ok
okok
ok

ok
ok
okokok


okokokokokokokokokok









ok
okokokok



ok
okokok


okok

ok
okok
okok


ok
okokokokokok





ok
okokok


okokokokok




ok
okokokokokokokok






ok

okok

ok
ok
okok

okok

okok
ok

okokok


okokokokokok





okokokokok




ok
okokokokok




ok
ok
ok
ok
ok
okokokokokok





okokok


okok

okok

ok
ok
ok
ok
okok

okokokok



ok
ok
ok
ok
okokokok



ok
ok
ok
okok

ok
ok
okokokok



ok
okokok

okokokokokokokokokokokokok












okok


ok
ok
ok
ok
okokok


okokokokokok


ok


ok

ok
okokok


okok
ok

okokokokok




okokokokok




okokokokok



ok
okok


okokok

ok